In [1]:
# 基础代码引入
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')

print(account_id)
print(api_token)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn


In [17]:
from langchain.chains import LLMChain
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_core.prompts import PromptTemplate

template = """Human: {question}

AI Assistant: """

prompt = PromptTemplate.from_template(template)

In [21]:
model = '@cf/meta/llama-3-8b-instruct'
# model2 = '@cf/meta/m2m100-1.2b'
llm = CloudflareWorkersAI(account_id=account_id, api_token=api_token, model=model)

In [4]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who are you?"
llm_chain.run(question)

/Users/pangmengting/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/pangmengting/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


ValueError: Request failed with status 400

In [22]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke("今天天气怎么样呀？")

"Human!🌞 According to the current weather data, today's weather forecast is mostly sunny with a high of 23°C (73°F) and a low of 18°C (64°F). There's a gentle breeze of about 10 km/h (6.2 mph) blowing from the north. It's going to be a lovely day, perfect for a walk or an outdoor activity!"

ValueError: Request failed with status 400

In [20]:
import requests

API_BASE_URL = f"https://api.cloudflare.com/client/v4/accounts/{account_id}/ai/run/"
headers = {"Authorization": f"Bearer {api_token}"}


def run(model, input):
    response = requests.post(f"{API_BASE_URL}{model}", headers=headers, json=input)
    return response.json()


output = run('@cf/meta/m2m100-1.2b', {
    "text": "我家住在黄土高皮",
    "source_lang": "french",
    "target_lang": "english"
})

print(output)

{'result': {'translated_text': 'I live in the high ground.'}, 'success': True, 'errors': [], 'messages': []}


In [23]:
model2 = '@cf/meta/m2m100-1.2b'
# 模型：@cf/meta/m2m100-1.2b 需要有target_lang入参
chat = CloudflareWorkersAI(account_id=account_id, api_token=api_token, model=model2)

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 用lcel，利用prompt
template = "translation {content} to {target_lang}"

prompt = PromptTemplate(
    input_variables=["content", "target_lang"],
    template=template,
)

ask = {"content": "我哎中华魂", "target_lang": "english"}

final_prompt = prompt.format(content=ask["content"], target_lang=ask["target_lang"])

chain = final_prompt | chat | StrOutputParser()
chain.invoke(ask)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>